In [4]:
%load_ext autoreload
%autoreload 2

import sys 
sys.path.append("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import tensorflow as tf
import numpy as np
#import keras
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D,Softmax
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy


In [6]:
# CIFAR10 Dataset
from modules.utils import load_data
X_train, y_train = load_data(mode='train')
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1) / 255
X_val = X_val.reshape(-1,3,32,32).transpose(0,2,3,1) / 255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

y_train_dummy = tf.keras.utils.to_categorical(y_train)
y_val_dummy = tf.keras.utils.to_categorical(y_val)
print('Train labels shape (one-hot): ', y_train_dummy.shape)
print('Validation labels shape (one-hot): ', y_val_dummy.shape)

Start downloading data...
Download complete.
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Train labels shape (one-hot):  (49000, 10)
Validation labels shape (one-hot):  (1000, 10)


In [8]:
from modules.layers import spectralConv2D


def spectralCNN(input_shape):

    model = Sequential()

    model.add(spectralConv2D(96, kernel_size=(3,3),padding="VALID", strides=(1,1), input_shape=input_shape))
    model.add(spectralConv2D(96, kernel_size=(3,3),padding="VALID", strides=(1,1) ))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

    model.add(spectralConv2D(192, kernel_size=(3,3),padding="VALID",strides=(1,1)))
    model.add(spectralConv2D(192, kernel_size=(3,3),padding="VALID", strides=(1,1)))
    model.add(spectralConv2D(192, kernel_size=(3,3),padding="VALID", strides=(1,1)))        
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    
    model.add(spectralConv2D(192, kernel_size=(1,1),padding="VALID",strides=(1,1)))

    model.add(spectralConv2D(10, kernel_size=(1,1),padding="VALID",strides=(1,1)))    
    model.add(GlobalAveragePooling2D())
    model.add(Softmax())
    return model
    


In [ ]:
spectral_cnn_model = spectralCNN(X_train.shape[1:])

spectral_cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history=spectral_cnn_model.fit(x=X_train, y=y_train,
          batch_size=128,
          epochs=25, 
          validation_data=(X_val, y_val)
              )

Epoch 1/25
383/383 [==============================] - 33s 32ms/step - loss: 2.0779 - accuracy: 0.2540 - val_loss: 1.7435 - val_accuracy: 0.4100
Epoch 2/25
383/383 [==============================] - 11s 29ms/step - loss: 1.6455 - accuracy: 0.4468 - val_loss: 1.4074 - val_accuracy: 0.5150
Epoch 3/25
383/383 [==============================] - 11s 30ms/step - loss: 1.3160 - accuracy: 0.5556 - val_loss: 1.2179 - val_accuracy: 0.5950
Epoch 4/25
383/383 [==============================] - 11s 30ms/step - loss: 1.1793 - accuracy: 0.5993 - val_loss: 1.1374 - val_accuracy: 0.6060
Epoch 5/25
383/383 [==============================] - 12s 30ms/step - loss: 1.0832 - accuracy: 0.6326 - val_loss: 1.1470 - val_accuracy: 0.6130
Epoch 6/25
383/383 [==============================] - 12s 30ms/step - loss: 1.0141 - accuracy: 0.6502 - val_loss: 1.0867 - val_accuracy: 0.6250
Epoch 7/25
383/383 [==============================] - 12s 31ms/step - loss: 0.9587 - accuracy: 0.6664 - val_loss: 1.0279 - val_accuracy:

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()